In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 25 15:57:40 2019

@author: agarc
"""
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics
import sklearn.preprocessing
from sklearn.feature_selection import VarianceThreshold
import os
import collections

In [2]:
class musicFan:
    
    keys = np.array(["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness"\
            ,"liveness","valence","tempo"])
    indices = [0,1,2,3,4,5,6,7,8,9,10]
    
    def __init__(self,ID,tastes):
        
        self.ID = ID
        self.session = np.int_(tastes[11])
        self.dataID = np.array(tastes[13],dtype = "str")
        #self.songID = tastes[14] #this has issues right now, always use dataID
        self.response = np.int_(tastes[12]) == 1 #Mask array of True or False where True is like and False is no like.
        self.tastes = np.array([np.float64(tastes[0]), np.float64(tastes[1]), np.int_(tastes[2])\
                       , np.float64(tastes[3]), np.int_(tastes[4]), np.float64(tastes[5]),\
                       np.float64(tastes[6]), np.float64(tastes[7]), np.float64(tastes[8])\
                       , np.float64(tastes[9]), np.float64(tastes[10])]).T
        self.songNames = []
        self.simSongsNames = []
        self.simSongsID = []
        
        if not os.path.exists("UserOutput"):
            os.makedirs("UserOutput")
        if not os.path.exists("UserOutput/User"+str(self.ID)):
            os.makedirs("UserOutput/User"+str(self.ID))
        
        
    def getFromData(self,dataBase):
        self.songNames = []
        songNames = np.array(dataBase["song_name"])
        dID = np.array(dataBase["database_id"])
        for i in range(len(self.dataID)):
            for x in range(len(songNames)):
                if dID[x] in self.dataID[i]:
                    self.songNames.append(songNames[x])
        self.songNames = np.array(self.songNames)

        
    def mean(self):
        self.mean_Tastes = np.mean(self.tastes, axis = 1)
        self.median_Tastes = np.median(self.tastes, axis = 1)
        self.percent_Positive = np.mean(self.response)
        print("For musicFan "+str(self.ID)+" the mean values are:\n",self.mean_Tastes,"\n","The median values are:\n"\
              ,self.median_Tastes,"\n","Percent positive: " +str(self.percent_Positive)+"\n")
        #return self.mean_Tastes

    def makeScatters(self):
        num_Comb = 55
        
        if not os.path.exists("UserOutput/User"+str(self.ID)+"/scatters"):
            os.makedirs("UserOutput/User"+str(self.ID)+"/scatters")
        
        combinations = []
        
        for i in self.indices:
            for j in self.indices:
                if [i,j] not in combinations and [j,i] not in combinations and i != j:
                    combinations.append([i,j])
        for i in combinations:
            fig1, ax1 = plt.subplots()
            ax1.scatter(self.tastes[i[0]][np.isin(self.response,0)],self.tastes[i[1]][np.isin(self.response,0)], color = "red")
            ax1.scatter(self.tastes[i[0]][np.isin(self.response,1)],self.tastes[i[1]][np.isin(self.response,1)], color = "blue")
            ax1.set_xlabel(self.keys[i[0]])
            ax1.set_ylabel(self.keys[i[1]])
            ax1.set_title(self.keys[i[0]]+"vs."+self.keys[i[1]])
            plt.savefig("UserOutput/User"+str(self.ID)+"/scatters/"+self.keys[i[0]]+"_"+self.keys[i[1]]+".png")
            plt.close("all")
            
        print(combinations)
        print(len(combinations))
        
    def makeHistos(self):
        
        if not os.path.exists("UserOutput/User"+str(self.ID)+"/histograms"):
            os.makedirs("UserOutput/User"+str(self.ID)+"/histograms")
        
        for i in self.indices:
            fig1, ax1 = plt.subplots()
            ax1.hist(self.tastes[i], bins = 20, color = "green")
            ax1.hist(self.tastes[i][np.isin(self.response,0)], bins = 20, color = "red")
            ax1.hist(self.tastes[i][np.isin(self.response,1)], bins = 20, color = "blue")
            ax1.set_xlabel(self.keys[i])
            ax1.set_ylabel("count")
            ax1.set_title("Histogram of "+self.keys[i]+" for user: "+str(self.ID))
            plt.savefig("UserOutput/User"+str(self.ID)+"/histograms/"+self.keys[i]+"_histogram"+".png")
            plt.close("all")
            
    def cosineSimilarity(self,songs):
        a = sklearn.preprocessing.normalize(self.tastes[self.response == 1],axis=0)
        b = sklearn.preprocessing.normalize(songs.features,axis=0)
        self.cSim = sklearn.metrics.pairwise.cosine_similarity(a,b)
        self.simSongsNames = np.array([songs.name[i>0.992] for i in self.cSim])
        self.simSongsID = np.array([songs.ID[i>0.992] for i in self.cSim])
        print("Cosinesimilarity successful for user:",self.ID,".")
    
    def crossValidate(userList):
        a=1

In [3]:
class songs:

    keys = ["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo"]
    
    def __init__(self,features):
        
        self.ID = features[:,18]
        self.name = features[:,19]
        self.albumID = features[:,2]
        self.albumName = features[:,3]
        self.artistID = features[:,5]
        self.artistName = features[:,6]
        self.year = features[:,26]
        self.features = np.array([np.float64(features[:,7]), np.float64(features[:,10]), np.float64(features[:,12]), np.float64(features[:,14]),\
                                    np.int_(features[:,15]), np.float64(features[:,20]), np.float64(features[:,1]),\
                                    np.float64(features[:,11]),np.float64(features[:,13]),np.float64(features[:,25]),np.float64(features[:,21])]).T
      
    def selectFeatures(self,variance):
        selection = VarianceThreshold(threshold = (variance))
        selection.fit_transform(self.features)
        return selection.get_support()
        #This will be code to select features based on variance. Return a mask for features.
    
    def recommend(self,user):
        print('hello')
        a = np.concatenate([self.ID[user.cSim[i]>0.97] for i in range(len(user.cSim))])
        b = collections.Counter(a).most_common() #returns a count of values        
        return b

In [4]:
"""
Created on Mon Feb 25 16:34:54 2019
@author: agarc
"""
#This file runs various methods defined in the spotifyProjectClasses.py module.
#Methods can be disabled by commenting lines.

#testing change

import numpy as np
import os,sys
#import spotifyProjectClasses as spc
import pandas as pd
import time

In [5]:
t0 = time.time()

#setting the working directory to file location
filepath = "C:"+str("\\")+"Users\Mukesh\Documents\Python Scripts\\2nd_sem_ML_Stat\spotify_data_for_students_by_ST_ML\spotify_data_for_students\\"#os.path.dirname(os.path.abspath(sys.argv[0]))
print(filepath)
os.chdir(filepath)


C:\Users\Mukesh\Documents\Python Scripts\2nd_sem_ML_Stat\spotify_data_for_students_by_ST_ML\spotify_data_for_students\


In [6]:
#opening all user data and music data
#############################################################
user_Filenames = os.listdir("user_data")
music_Filenames = os.listdir("music_data")
print(music_Filenames,"\n")
#############################################################
print(user_Filenames,"\n")

data_Base = pd.read_csv("user_data/database.csv",delimiter = ",", dtype = "str")
#############################################################
data_Base.head()

['1991.csv', '1992.csv', '1993.csv', '1994.csv', '1995.csv', '1996.csv', '1997.csv', '1998.csv', '1999.csv', '2000.csv', '2001.csv', '2002.csv', '2003.csv', '2004.csv', '2005.csv', '2006.csv', '2007.csv', '2008.csv', '2009.csv', '2010.csv', '2011.csv', '2012.csv', '2013.csv', '2014.csv', '2015.csv', '2016.csv', '2017.csv', '2018.csv'] 

['.DS_Store', '.ipynb_checkpoints', 'alpha100.csv', 'alpha50.csv', 'alpha75.csv', 'database.csv'] 



,database_id,spotify_id,song_name,preview_url,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,5bb7ac890c27ee4c381dd69e,0bxllxzqkGNSKo8pHtZyA3,E.T.,https://p.scdn.co/mp3-preview/b010313e8fbd71b1...,0.626,0.457,8,-9.091,0,0.0339,0.0876,7.72E-05,0.0801,0.223,149.99
1,5bb7ac890c27ee4c381dd69f,362LzQDbfNdZDuv37BIEkG,Song For Zula - Live From Spotify London,https://p.scdn.co/mp3-preview/f85a3283dc9d614c...,0.441,0.344,4,-9.799,1,0.0299,0.386,0.000277,0.917,0.111,87.588
2,5bb7ac890c27ee4c381dd6a0,3AqPL1n1wKc5DVFFnYuJhp,To Build A Home,https://p.scdn.co/mp3-preview/b39e590a07f73f93...,0.264,0.128,9,-15.443,1,0.034,0.885,0.352,0.105,0.0774,148.499
3,5bb7ac890c27ee4c381dd6a1,7sDzAu7alAL1hWvL86tSGB,Sweater Weather - Spotify Sessions Curated by ...,https://p.scdn.co/mp3-preview/bed9e182c4c5af6b...,0.61,0.165,8,-14.659,1,0.0502,0.981,0.000544,0.134,0.258,102.68
4,5bb7ac890c27ee4c381dd6a2,7LVHVU3tWfcxj5aiPFEW4Q,Fix You,https://p.scdn.co/mp3-preview/b1640815319b2df5...,0.209,0.418,3,-8.74,1,0.0338,0.163,0.00195,0.113,0.123,138.265


In [7]:
#user_Files = [pd.read_csv("user_data/"+i,delimiter = ",", dtype = "str") for i in user_Filenames if "alpha" in i]
#merging 3 different distributed files and creating a single dataframe
df_50 = pd.read_csv(r"C:\Users\Mukesh\Documents\Python Scripts\2nd_sem_ML_Stat\spotify_data_for_students_by_ST_ML\spotify_data_for_students\user_data\alpha50.csv")
df_75 = pd.read_csv(r"C:\Users\Mukesh\Documents\Python Scripts\2nd_sem_ML_Stat\spotify_data_for_students_by_ST_ML\spotify_data_for_students\user_data\alpha75.csv")
df_100 = pd.read_csv(r"C:\Users\Mukesh\Documents\Python Scripts\2nd_sem_ML_Stat\spotify_data_for_students_by_ST_ML\spotify_data_for_students\user_data\alpha100.csv")
temp=[df_50,df_75,df_100]
user_Files=pd.concat(temp,ignore_index=True)
user_Files.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,user_session,user_response,user_id,database_id
0,0.327,0.6860,10,-11.835,0,0.1610,0.16300,0.194,0.9480,0.631,172.790,1,1,0,5bb7ac8a0c27ee4c381dd70a
1,0.659,0.5880,6,-9.262,0,0.0300,0.41300,0.159,0.1120,0.561,143.021,1,0,0,5bb7ac8a0c27ee4c381dd70d
2,0.333,0.0797,10,-14.597,1,0.0360,0.97000,0.000,0.1120,0.221,176.535,1,0,0,5bb7ac930c27ee4c381dd9c7
3,0.367,0.7690,9,-7.258,1,0.0317,0.00032,0.628,0.0631,0.700,173.186,1,1,0,5bb7ac8a0c27ee4c381dd703
4,0.443,0.8510,7,-3.809,1,0.2180,0.08560,0.000,0.4990,0.693,170.346,1,1,0,5bb7ac9a0c27ee4c381ddbab


In [8]:
#for i in range(len(user_Files)):
user_Files["spotify_id"] = data_Base["spotify_id"][data_Base["database_id"].isin(user_Files["database_id"])]
user_Data = user_Files.values
print(user_Data[0])
print(type(user_Data))
user_Data.shape
user_Files.loc[:,'spotify_id'].head()

[0.327 0.6859999999999999 10 -11.835 0 0.161 0.163 0.19399999999999998
 0.948 0.631 172.79 1 1 0 '5bb7ac8a0c27ee4c381dd70a'
 '0bxllxzqkGNSKo8pHtZyA3']
<class 'numpy.ndarray'>


0    0bxllxzqkGNSKo8pHtZyA3
1    362LzQDbfNdZDuv37BIEkG
2    3AqPL1n1wKc5DVFFnYuJhp
3    7sDzAu7alAL1hWvL86tSGB
4    7LVHVU3tWfcxj5aiPFEW4Q
Name: spotify_id, dtype: object

In [9]:
#############################################################
music_Files = [pd.read_csv("music_data/"+i,delimiter = ",", dtype = "str") for i in music_Filenames]
for i in range(len(music_Files)):
    music_Files[i]["year"] = music_Filenames[i][:-4]
music_Data = pd.concat(music_Files).values
#############################################################

In [10]:
t1 = time.time()
#initializing a set of user objects based on the musicFan class
m_Fans = []
print('music_Data=',music_Data[np.isfinite(np.float64(music_Data[:,7]))])
print(type(music_Data[np.isfinite(np.float64(music_Data[:,7]))]))
#adding songs to songs object, contains all songs
songs_list = songs(music_Data[np.isfinite(np.float64(music_Data[:,7]))])
#This loop creates a list of m_Fan objects.
t1 = time.time()
for i in range(10):
    temp1 = [[user_Data[x][y] for x in range(len(user_Data)) if i ==user_Data[x][13]] for y in range(16) if y !=12]
    print('temp1=',type(temp1))
    m_Fans.append(musicFan(i,temp1))
t2 = time.time()
print(t2-t1, "seconds for making user objects")
print(m_Fans[0].tastes.shape)

music_Data= [['0' '0.108' '67nqYhvjDyJXGD64937Lw3' ... '1' '0.971' '1991']
 ['1' '0.425' '4S2ZR5njbJez1rV8DV5Vtl' ... '2' '0.639' '1991']
 ['2' '0.0468' '0IyCnBPnQXafGgV0Jsa5Kq' ... '4' '0.859' '1991']
 ...
 ['9312' '0.00158' '7KTAGt9DgZpqrqtRu6RRXz' ... '1' '0.252' '2018']
 ['9313' '0.000768' '5knqLpyyuMFpNhEdTfYDcG' ... '41' '0.788' '2018']
 ['9314' '0.00257' '3Op5tIoPMiBgfJ9VSm3nUC' ... '1' '0.494' '2018']]
<class 'numpy.ndarray'>
temp1= <class 'list'>
temp1= <class 'list'>
temp1= <class 'list'>
temp1= <class 'list'>
temp1= <class 'list'>
temp1= <class 'list'>
temp1= <class 'list'>
temp1= <class 'list'>
temp1= <class 'list'>
temp1= <class 'list'>
0.26575660705566406 seconds for making user objects
(338, 11)


In [11]:
#------------------------------------------------------------------------------#
#this pulls additional info from the database file    
t1 = time.time()
for i in range(10):
    m_Fans[i].getFromData(data_Base) 
t2 = time.time()
print(t2-t1, "seconds for getting data from database")


#------------------------------------------------------------------------------#  
#this applies cosinesimilarity method between each user and complete song list  
t1 = time.time()
for i in range(10):
    m_Fans[i].cosineSimilarity(songs_list) #users have a variable called cSim which is the output of cosineSimilarity:
t2 = time.time()
print(t2-t1, "seconds for applying cosine similarity")


#------------------------------------------------------------------------------# 
#This returns a list of recommended songs for each user.

a = songs.recommend(m_Fans[0])

'''
#for example:
#mFan 0 has a cSim matrix of shape 109x265119:
#this is because User0 liked 109 songs. Each song has an associated matrix of length 265119 attached to it.
print(m_Fans[0].cSim.shape)
#this matrix has values ranging from 0-1 where 0 is not similar and 1 is identical.
#we can find all songs that are very similar to the suer-liked song by using a mask:
print(songs.name[m_Fans[0].cSim[0]>0.992])   
#this prints out the names of songs whose features are similar to those of song0 in user0
#we can probably improve this by removing features that have little effect on the actual sound. I have tested some of these by comparing the songs on youtube
#and it seems to work decently.
'''   


3.6474437713623047 seconds for getting data from database


ValueError: Found array with 0 sample(s) (shape=(0, 11)) while a minimum of 1 is required by the normalize function.